# MoneyGram Lend Bot

Link can be found [here](https://colab.research.google.com/drive/1AccxMSblKWakuADJb9IhgMzVwrYojKuA?usp=sharing)



In [ ]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics



from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier


from imblearn.over_sampling import RandomOverSampler
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

# Clean and Prepare Data for ML

In [ ]:
# Import Credit Risk information to Google Colab
url = 'https://raw.githubusercontent.com/DIsaacman/Dream-Team/dataset/Resources/credit_risk_dataset.csv'

# Read CSV
df = pd.read_csv(url)

# Review the DataFrame
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [ ]:
# Assess information
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  32581 non-null  int64  
 1   person_income               32581 non-null  int64  
 2   person_home_ownership       32581 non-null  object 
 3   person_emp_length           31686 non-null  float64
 4   loan_intent                 32581 non-null  object 
 5   loan_grade                  32581 non-null  object 
 6   loan_amnt                   32581 non-null  int64  
 7   loan_int_rate               29465 non-null  float64
 8   loan_status                 32581 non-null  int64  
 9   loan_percent_income         32581 non-null  float64
 10  cb_person_default_on_file   32581 non-null  object 
 11  cb_person_cred_hist_length  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB


None

In [ ]:
# Drop Columns

consolidated_df = df.drop(columns=[
       'person_emp_length', 'loan_intent', 'loan_grade',
       'loan_int_rate', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length'])

In [ ]:
# Create a list of categorical variables 
categorical_variables = list(consolidated_df.dtypes[consolidated_df.dtypes=="object"].index)

# Display the categorical variables list
categorical_variables

['person_home_ownership']

## Use OneHotEncoder to convert categorical variable

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [ ]:
#Fit The model
encoded_data = enc.fit_transform(consolidated_df[categorical_variables])

In [ ]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
columns=enc.get_feature_names_out(categorical_variables)
)

# Review the DataFrame
encoded_df.head()

,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT
0,0.0,0.0,0.0,1.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0


In [ ]:
# Create a new df that does not include those columns we converted with one_hot_encoder (all numerical variables from original dataset)
numerical_variables_df = consolidated_df.drop(columns=categorical_variables)

#review the dataframe
numerical_variables_df.head()

,person_age,person_income,loan_amnt,loan_status
0,22,59000,35000,1
1,21,9600,1000,0
2,25,9600,5500,1
3,23,65500,35000,1
4,24,54400,35000,1


In [ ]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = pd.concat([encoded_df, numerical_variables_df], axis=1)

# Review the Dataframe
encoded_df.head()

,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,person_age,person_income,loan_amnt,loan_status
0,0.0,0.0,0.0,1.0,22,59000,35000,1
1,0.0,0.0,1.0,0.0,21,9600,1000,0
2,1.0,0.0,0.0,0.0,25,9600,5500,1
3,0.0,0.0,0.0,1.0,23,65500,35000,1
4,0.0,0.0,0.0,1.0,24,54400,35000,1


### Define Feature and Target Set

In [ ]:
# Separate the data into features and target
y = encoded_df["loan_status"]
y[:5]

0    1
1    0
2    1
3    1
4    1
Name: loan_status, dtype: int64

In [ ]:
# Define Feature set
X = encoded_df.drop(columns="loan_status")
X.sample(5)

,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,person_age,person_income,loan_amnt
5475,0.0,0.0,0.0,1.0,24,60000,5000
8705,0.0,0.0,0.0,1.0,24,35000,8000
14745,1.0,0.0,0.0,0.0,24,113676,7200
24138,1.0,0.0,0.0,0.0,32,70000,14000
21383,0.0,0.0,1.0,0.0,33,48000,14400


In [ ]:
# Check the value_counts to see whether there is an oversampling issue
y.value_counts()

0    25473
1     7108
Name: loan_status, dtype: int64

## Split The Data using train_test_split

In [ ]:
# Split the data using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Utilize RandomOverSampler

In [ ]:
# Instantiate the random oversampler model
random_oversampler = RandomOverSampler(random_state=1)

# Fit the training data to the random_oversampler model
X_resampled, y_resampled = random_oversampler.fit_resample(X_train, y_train)

In [ ]:
# Check the value_counts to see whether there is an oversampling issue
y_resampled.value_counts()

0    19113
1    19113
Name: loan_status, dtype: int64

### Scale the Data with StandardScaler

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## 1. Logistic Regression Model

In [ ]:
# Instantiate the Logistic Regression model
logistic_regression_model = LogisticRegression(random_state=1)
# Fit the model using the resampled training data
lr_model = logistic_regression_model.fit(X_resampled, y_resampled)
# Make a prediction using the testing data
predictions_oversampled = logistic_regression_model.predict(X_test)

In [ ]:
predictions_oversampled

array([0, 1, 0, ..., 0, 0, 1])

In [ ]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: loan_status, dtype: int64

In [ ]:
# X_test.columns

user_test = pd.DataFrame(columns = ['person_home_ownership_MORTGAGE', 'person_home_ownership_OTHER',
       'person_home_ownership_OWN', 'person_home_ownership_RENT', 'person_age',
       'person_income', 'loan_amnt'])

test=1 
user_test['person_home_ownership_MORTGAGE'] = [0]
user_test['person_home_ownership_OTHER'] = [0]
user_test['person_home_ownership_OWN'] = [1]
user_test['person_home_ownership_RENT'] = [0]
user_test['person_age'] = [21]
user_test['person_income'] = [9600]
user_test['loan_amnt'] = [1000]


# y.head()

X_test

# Make a prediction using the testing data
predictions_oversampled_user = logistic_regression_model.predict(user_test)
predictions_oversampled_user

# user_test

array([0, 1, 0, ..., 0, 0, 1])

Results

In [ ]:
#Get accuracy score from the testing data
balanced_accuracy_score(y_test, predictions_oversampled)

0.6767254220456802

In [ ]:
#Generate Confusion Matrix
confusion_matrix(y_test, predictions_oversampled)

array([[4089, 2271],
       [ 517, 1269]])

In [ ]:
#Show Classification Report
print(classification_report_imbalanced(y_test, predictions_oversampled))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.89      0.64      0.71      0.75      0.68      0.45      6360
          1       0.36      0.71      0.64      0.48      0.68      0.46      1786

avg / total       0.77      0.66      0.70      0.69      0.68      0.46      8146



## 2. BalancedRandomForestClassifier Model

In [ ]:
# Instantiate a BalancedRandomForestClassifier instance
brf = BalancedRandomForestClassifier()

# Fit the model to the training data
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier()

In [ ]:
# Predict labels for testing features
y_pred = brf.predict(X_test_scaled)

In [ ]:
# Print classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.82      0.86      6360
           1       0.52      0.71      0.60      1786

    accuracy                           0.79      8146
   macro avg       0.71      0.76      0.73      8146
weighted avg       0.82      0.79      0.80      8146



In [ ]:
#Get accuracy score from the testing data
balanced_accuracy_score(y_test, y_pred)

0.7612092128152577

## 3. KNN Model (K Nearest Neighbor)

In [ ]:
#Initiate KNN model with cluster of 3
knn_model = KNeighborsClassifier(n_neighbors=3) 

In [ ]:
#Fit the Model
knn_model.fit(X_train_scaled, y_train) 

KNeighborsClassifier(n_neighbors=3)

In [ ]:
 # Create predictions
y_pred_knn = knn_model.predict(X_test_scaled)

# Review the predictions
y_pred_knn

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
#Print Classification Report
print(classification_report(y_pred_knn,y_test)) 

              precision    recall  f1-score   support

           0       0.93      0.86      0.89      6862
           1       0.47      0.65      0.54      1284

    accuracy                           0.83      8146
   macro avg       0.70      0.76      0.72      8146
weighted avg       0.86      0.83      0.84      8146



In [ ]:
#Get accuracy score from the testing data
balanced_accuracy_score(y_pred_knn,y_test)

0.7553988238082137

----------

### 4. Adaboost Model

In [ ]:
# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=100,
                         learning_rate=1)

In [ ]:
# Train Adaboost Classifer
abc_model = abc.fit(X_train_scaled, y_train)

In [ ]:
#Predict the response for test dataset
abc_pred = abc_model.predict(X_test_scaled)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, abc_pred))

In [ ]:
print(classification_report(y_test, abc_pred)) 

In [ ]:
balanced_accuracy_score(abc_pred,y_test)